In [1]:
import numpy as np 

import pandas as pd 
import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

In [2]:
website = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
page = requests.get(website)

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of areas of London - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"63cda421-6640-4415-8f57-f92dbb0ab4c5","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_areas_of_London","wgTitle":"List of areas of London","wgCurRevisionId":963608799,"wgRevisionId":963608799,"wgArticleId":11915713,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Use dmy dates from August 2015","Use British English from August 2015","Lists of coord

In [4]:
text = soup.get_text()
split = text.split('\n')
split = list(filter(None, split))
split

['List of areas of London - Wikipedia',
 'List of areas of London',
 'From Wikipedia, the free encyclopedia',
 'Jump to navigation',
 'Jump to search',
 'Wikipedia list article',
 '"Districts of London" and "List of districts of London" redirect here. For the local authority districts in London, see London boroughs and List of London boroughs.',
 'London is the capital of and largest city in England and the United Kingdom. It is administered by the City of London and 32 London boroughs. These boroughs are modern, having been created in 1965 and have a weaker sense of identity than their constituent "districts" (considered in speech, "parts of London" or more formally, "areas"). Two major factors have shaped the development of London district and sub-district identities; the Ancient Parish - which was used for both civil and ecclesiastical functions - and the pre-urban settlement pattern. ',
 'Contents',
 '1 Ancient Parishes and their successors',
 '2 Pre-urban settlement pattern',
 '3 

In [5]:
columns = 'Borough','Neighborhood','Latitude','Longitude'
df = pd.DataFrame(columns = columns)
df

,Borough,Neighborhood,Latitude,Longitude


In [6]:
for i in range(len(split)):
    if(split[i]=='Abbey Wood'):
        start = i
    if(split[i]=='TQ063804'):
        stop=i
        break
split = split[start:stop]
split

['Abbey Wood',
 'Bexley,  Greenwich [7]',
 'LONDON',
 'SE2',
 '020',
 'TQ465785',
 'Acton',
 'Ealing, Hammersmith and Fulham[8]',
 'LONDON',
 'W3, W4',
 '020',
 'TQ205805',
 'Addington',
 'Croydon[8]',
 'CROYDON',
 'CR0',
 '020',
 'TQ375645',
 'Addiscombe',
 'Croydon[8]',
 'CROYDON',
 'CR0',
 '020',
 'TQ345665',
 'Albany Park',
 'Bexley',
 'BEXLEY, SIDCUP',
 'DA5, DA14',
 '020',
 'TQ478728',
 'Aldborough Hatch',
 'Redbridge[9]',
 'ILFORD',
 'IG2',
 '020',
 'TQ455895',
 'Aldgate',
 'City[10]',
 'LONDON',
 'EC3',
 '020',
 'TQ334813',
 'Aldwych',
 'Westminster[10]',
 'LONDON',
 'WC2',
 '020',
 'TQ307810',
 'Alperton',
 'Brent[11]',
 'WEMBLEY',
 'HA0',
 '020',
 'TQ185835',
 'Anerley',
 'Bromley[11]',
 'LONDON',
 'SE20',
 '020',
 'TQ345695',
 'Angel',
 'Islington[8]',
 'LONDON',
 'EC1, N1',
 '020',
 'TQ345665',
 'Aperfield',
 'Bromley[11]',
 'WESTERHAM',
 'TN16',
 '01959',
 'TQ425585',
 'Archway',
 'Islington[12]',
 'LONDON',
 'N19',
 '020',
 'TQ285875',
 'Ardleigh Green',
 'Havering[12]',


In [7]:
def clean(string):
    s=''
    for i in string:
        if(i=='[' or i=='('):
            return s
        s = s+i
    return s
print(clean('City([10]'))

City


In [8]:
L=[]

count = 0 
for i in range(len(split)):
    if(i%6==1):
        L.append((clean(split[i]),clean(split[i-1])))
    if(split[i-1]=='Blendon'):
        count = i+4
        break
splitt = split[count:]
for i in range(len(splitt)):
    if(i%6==1):
        L.append((clean(splitt[i]),clean(splitt[i-1])))
    if(splitt[i-1]=='Hazelwood'):
        count = i+4
        break
splittt = splitt[count:]
#print(splittt[0])
for i in range(len(splittt)):
    if(i%6==1):
        L.append((clean(splittt[i]),clean(splittt[i-1])))
print(L[0:5])

[('Bexley,  Greenwich ', 'Abbey Wood'), ('Ealing, Hammersmith and Fulham', 'Acton'), ('Croydon', 'Addington'), ('Croydon', 'Addiscombe'), ('Bexley', 'Albany Park')]


In [9]:
count = 0
geolocator = Nominatim(user_agent="London")

for i in L:
    address = i[1] + ',' + i[0].split(',')[0] + ', London'
    print(address)
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        df.loc[count] =(i[0].split(',')[0],i[1],latitude,longitude)
        count+=1
    except:
        try:
            address = i[1] + ',' + i[0].split(',')[1]
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            df.loc[count] = (i[0].split(',')[0],i[1],latitude,longitude)
            count+=1
        except:
            pass


Abbey Wood,Bexley, London
Acton,Ealing, London
Addington,Croydon, London
Addiscombe,Croydon, London
Albany Park,Bexley, London
Aldborough Hatch,Redbridge, London
Aldgate,City, London
Aldwych,Westminster, London
Alperton,Brent, London
Anerley,Bromley, London
Angel,Islington, London
Aperfield,Bromley, London
Archway,Islington, London
Ardleigh Green,Havering, London
Arkley,Barnet, London
Arnos Grove,Enfield, London
Balham,Wandsworth, London
Bankside,Southwark, London
Barbican,City, London
Barking,Barking and Dagenham, London
Barkingside,Redbridge, London
Barnehurst,Bexley, London
Barnes,Richmond upon Thames, London
Barnes Cray,Bexley, London
Barnet Gate,Barnet, London
Barnet ,Barnet, London
Barnsbury,Islington, London
Battersea,Wandsworth, London
Bayswater,Westminster, London
Beckenham,Bromley, London
Beckton,Newham, London
Becontree,Barking and Dagenham, London
Becontree Heath,Barking and Dagenham, London
Beddington,Sutton, London
Bedford Park,Ealing, London
Belgravia,Westminster, London

Limehouse,Tower Hamlets, London
Lisson Grove,Westminster, London
Little Ilford,Newham, London
Little Venice,Westminster, London
Locksbottom,Bromley, London
Longford,Hillingdon, London
Longlands,Bexley, London
Lower Clapton,Hackney, London
Lower Morden,Merton, London
Loxford,Redbridge, London
Maida Vale,Westminster, London
Malden Rushett,Kingston upon Thames, London
Manor House,Hackney, London
Manor Park,Newham, London
Marks Gate,Barking and Dagenham, London
Maryland,Newham, London
Marylebone ,Westminster, London
Mayfair,Westminster, London
Maze Hill,Greenwich, London
Merton Park,Merton, London
Middle Park,Greenwich, London
Mile End,Tower Hamlets, London
Mill Hill,Barnet, London
Millbank,Westminster, London
Millwall,Tower Hamlets, London
Mitcham,Merton, London
Monken Hadley,Barnet, London
Morden,Merton, London
Morden Park,Merton, London
Mortlake,Richmond upon Thames, London
Motspur Park,Kingston upon Thames, London
Mottingham,Bromley, London
Muswell Hill,Haringey and Barnet, London
Nag'

In [10]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bexley,Abbey Wood,51.489962,0.132891
1,Ealing,Acton,51.508140,-0.273261
2,Croydon,Addington,51.358637,-0.031635
3,Croydon,Addiscombe,51.379692,-0.074282
4,Bexley,Albany Park,51.435384,0.125965


In [11]:
address = 'London, UK'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 51.5073219, -0.1276474.


In [12]:
maptor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maptor)  
    
maptor

In [13]:
t = pd.read_excel('HistoricSummary.xlsx',skiprows=1)
t

,Unnamed: 0,Unnamed: 1,Unnamed: 2,% Excellent or Good,% Strongly Agree or Tend to agree,% Strongly Agree or Tend to agree.1,% Strongly Agree or Tend to agree.2,% Strongly Agree or Tend to agree.3,% Very or Fairly informed,% Yes,...,% Well to some extent.1,% Not a problem or dont know,% Not very or not at all worried,% Not very or not at all worried.1,% Safe,% Well to some extent.2,% Satisfied,% Strongly Agree or Tend to agree.5,% Strongly Agree or Tend to agree.6,Unnamed: 21
0,BarkingandDagenham_Barking,KG-KB,Group 7,61.742424,71.647510,79.850746,85.440613,80.916031,37.358491,18.352060,...,62.745098,80.544747,69.662921,65.789474,65.502183,65.917603,78.030303,87.832700,78.651685,183.909626
1,BarkingandDagenham_Dagenham,KG-KR,Group 9,59.566787,67.266187,69.257951,84.751773,72.924188,45.035461,13.986014,...,58.545455,80.286738,60.701754,67.368421,66.000000,54.063604,76.140351,88.928571,73.118280,156.308107
2,BarkingandDagenham_Whalebone,KG-KK,Group 9,60.944206,70.085470,77.405858,80.508475,71.982759,38.559322,17.154812,...,60.606061,80.341880,63.598326,61.764706,63.000000,52.301255,86.134454,87.763713,72.457627,152.120018
3,Barnet_Barnet,SX-SA,Group 4,65.979381,64.285714,76.119403,77.435897,70.103093,50.738916,25.123153,...,48.538012,96.534653,70.935961,67.980296,79.787234,35.353535,89.603960,88.442211,72.680412,111.876831
4,Barnet_Colindale,SX-SC,Group 12,72.222222,72.277228,77.832512,80.500000,73.762376,44.334975,17.647059,...,50.000000,86.764706,68.627451,55.882353,77.894737,50.738916,86.138614,89.603960,74.371859,128.388670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Wandsworth_Putney,WW-WP,Group 1,75.789474,79.007634,83.972125,82.962963,82.031250,38.596491,11.564626,...,65.909091,85.159011,77.210884,80.677966,88.297872,60.068259,87.671233,92.000000,86.764706,142.926934
102,Wandsworth_Tooting,WW-WD,Group 1,73.362445,72.549020,81.171548,77.927928,75.369458,32.034632,14.979757,...,54.502370,79.146919,74.493927,81.300813,86.222222,52.282158,88.429752,90.134529,80.092593,149.950528
103,Westminster_Central,CW-CA,Group 2,77.439024,84.177215,83.435583,83.018868,86.000000,44.171779,9.467456,...,70.945946,91.139241,76.331361,82.738095,91.719745,67.272727,89.221557,92.073171,87.421384,363.558733
104,Westminster_North,CW-CM,Group 11,67.889908,81.388013,76.946108,83.072100,79.233227,42.378049,10.650888,...,66.229508,76.851852,72.271386,74.852071,85.064935,57.784431,86.309524,90.402477,81.875000,153.064101


In [14]:
df[df['Borough']=='City']

,Borough,Neighborhood,Latitude,Longitude
5,City,Aldgate,51.514248,-0.075719
17,City,Barbican,51.520150,-0.098683
48,City,Blackfriars,51.511585,-0.103767
447,City,Temple,51.510966,-0.114335


In [15]:
df_crime = pd.read_csv('crimerates.csv')
df_crime_2017 = df_crime[df_crime['Year']=='2016-17'].dropna()
df_crime_2017['Borough'].unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
       'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster',
       'Inner London', 'Outer London', 'Met Police Area'], dtype=object)

In [16]:
set(df['Borough'].unique().tolist()).difference(set(df_crime_2017['Borough'].unique().tolist()))

{'Camden and Islington',
 'City',
 'Dartford',
 'Haringey and Barnet',
 'Islington & City'}

In [17]:
set(df_crime_2017['Borough'].unique().tolist()).difference(set(df['Borough'].unique().tolist()))

{'Inner London', 'Met Police Area', 'Outer London'}

In [144]:
newdf.to_csv('newdf.csv')

In [18]:
df['Borough'] = df['Borough'].replace({'Camden and Islington':'Camden','Islington & City':'Islington','Dartford':'Bexley','Haringey and Barnet':'Haringey','City':'Westminster'})
df['Borough'].unique()

array(['Bexley', 'Ealing', 'Croydon', 'Westminster', 'Brent', 'Bromley',
       'Islington', 'Havering', 'Barnet', 'Enfield', 'Wandsworth',
       'Southwark', 'Barking and Dagenham', 'Redbridge',
       'Richmond upon Thames', 'Newham', 'Sutton', 'Lewisham', 'Harrow',
       'Camden', 'Kingston upon Thames', 'Tower Hamlets', 'Greenwich',
       'Haringey', 'Hounslow', 'Lambeth', 'Kensington and Chelsea',
       'Waltham Forest', 'Merton', 'Hackney', 'Hillingdon',
       'Hammersmith and Fulham'], dtype=object)

In [19]:
len(df['Borough'].unique())

32

In [278]:
CLIENT_ID = 'PLSZ3ZALVZMJGGF4T3KDWNEYM50TJYI2AEFRMPFM50KEQZ3H' # your Foursquare ID
CLIENT_SECRET = 'YU00DCXPHWP5J3ACL5X5VNA50BOP5PYX0TA2I40DJHBUKYGL' # your Foursquare Secret
VERSION = '20181224' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: PLSZ3ZALVZMJGGF4T3KDWNEYM50TJYI2AEFRMPFM50KEQZ3H
CLIENT_SECRET:YU00DCXPHWP5J3ACL5X5VNA50BOP5PYX0TA2I40DJHBUKYGL


In [20]:
dfcrime = df_crime_2017[df_crime_2017['Offences']=='All recorded offences']
t = ['Inner London','Outer London','Met Police Area']
dfcrime = dfcrime[~dfcrime['Borough'].isin(t)]
dfcrime.reset_index(inplace=True)
dfcrime.drop(['Code','Year','Offences','index','Number_of_offences'],axis=1,inplace=True)
dfcrime.head()


,Borough,Rate
0,Barking and Dagenham,70.9
1,Barnet,55.8
2,Bexley,45.6
3,Brent,70.8
4,Bromley,52.7


In [21]:
newdf = pd.merge(df,dfcrime,on='Borough')
newdf

,Borough,Neighborhood,Latitude,Longitude,Rate
0,Bexley,Abbey Wood,51.489962,0.132891,45.6
1,Bexley,Albany Park,51.435384,0.125965,45.6
2,Bexley,Barnehurst,51.465021,0.159647,45.6
3,Bexley,Barnes Cray,51.453299,0.192024,45.6
4,Bexley,Belvedere,51.489474,0.147359,45.6
...,...,...,...,...,...
516,Hammersmith and Fulham,Sands End,51.472545,-0.187112,98.1
517,Hammersmith and Fulham,Shepherd's Bush,51.505314,-0.222901,98.1
518,Hammersmith and Fulham,West Kensington,51.490662,-0.205916,98.1
519,Hammersmith and Fulham,White City,51.511935,-0.224236,98.1


In [22]:
housingdf = pd.read_csv('housing.csv')
housingdf = housingdf[housingdf['Year']=='Year ending Dec 2017']
housingdf = housingdf[housingdf['Measure']=='Mean']
len(set(housingdf['Ward_name'].tolist()).difference(set(newdf['Neighborhood'].tolist())))


419

In [23]:
G = newdf['Neighborhood'].tolist()
G.sort()


In [24]:
print(len(G),)

521


In [25]:
k = housingdf.shape
k

(630, 6)

In [26]:
CLIENT_ID = 'GYV2CKD0RBMJPI1IPNQLCR3PMPHMPXWTPP30PENLZRG4OA2Z'
CLIENT_SECRET = '05MQ1RTRYJRSGL4BIWU0GCXY2TQPTHY5Z2DR1R2MC5TH4AE2' # your Foursquare Secret
VERSION = '20202206' # Foursquare API version

print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT=100

credentails:
CLIENT_ID: GYV2CKD0RBMJPI1IPNQLCR3PMPHMPXWTPP30PENLZRG4OA2Z
CLIENT_SECRET:05MQ1RTRYJRSGL4BIWU0GCXY2TQPTHY5Z2DR1R2MC5TH4AE2


In [145]:
def getNearbyVenues(borough,names, latitudes, longitudes, radius=500):
    import requests
    venues_list=[]
    for boro,name, lat, lng in zip(borough,names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
           #print(results[0].keys())
            venues_list.append([(
            boro,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            print(venue_list[0])
        except:
            pass
        # return only relevant information for each nearby venue
        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
vdf = getNearbyVenues(newdf['Borough'],newdf['Neighborhood'], newdf['Latitude'], newdf['Longitude'], radius=500)

Abbey Wood
Albany Park
Barnehurst
Barnes Cray
Belvedere
Bexley 
Bexleyheath 
Blackfen
Blendon
Colyers
Crayford
Crook Log
Crossness
Dartford
East Wickham
Erith
Falconwood
Foots Cray
Lamorbey
Lessness Heath
Longlands
North Cray
Northumberland Heath
Ruxley
Sidcup
Slade Green
Thamesmead
Upper Ruxley
Upton
Welling
West Heath
Acton
Bedford Park
Ealing
Greenford
Hanwell
Northolt
Norwood Green
Perivale
Southall
West Ealing
Addington
Addiscombe
Coombe
Croydon
Forestdale
Kenley
New Addington
Norbury
Old Coulsdon
Purley
Riddlesdown
Sanderstead
Selhurst
Selsdon
Shirley
South Croydon
South Norwood
Thornton Heath
Upper Norwood
Waddon
Woodside
Aldgate
Aldwych
Barbican
Bayswater
Belgravia
Blackfriars
Charing Cross
Chinatown
Knightsbridge
Lisson Grove
Little Venice
Maida Vale
Marylebone 
Mayfair
Millbank
Paddington
Pimlico
Soho
St James's
St John's Wood
Temple
Westminster
Alperton
Brent Park
Brondesbury
Church End
Dollis Hill
Harlesden
Kensal Green
Kenton
Kilburn
Kingsbury
Neasden
Park Royal
Preston
Qu

In [29]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            51.588122, 
            -0.059937, 
            500, 
            LIMIT)

In [30]:
results = requests.get(url).json()["response"]['groups'][0]['items']

In [162]:
V500.columns.tolist()

['ATM',
 'Accessories Store',
 'Adult Boutique',
 'Afghan Restaurant',
 'African Restaurant',
 'Airport',
 'Airport Service',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Aquarium',
 'Arcade',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Arts & Entertainment',
 'Asian Restaurant',
 'Athletics & Sports',
 'Australian Restaurant',
 'Austrian Restaurant',
 'Auto Dealership',
 'Auto Garage',
 'Auto Workshop',
 'Automotive Shop',
 'BBQ Joint',
 'Badminton Court',
 'Bagel Shop',
 'Bakery',
 'Bar',
 'Baseball Field',
 'Bay',
 'Beach',
 'Bed & Breakfast',
 'Beer Bar',
 'Beer Garden',
 'Beer Store',
 'Belgian Restaurant',
 'Betting Shop',
 'Bike Rental / Bike Share',
 'Bike Shop',
 'Bike Trail',
 'Bistro',
 'Boarding House',
 'Boat or Ferry',
 'Bookstore',
 'Botanical Garden',
 'Boutique',
 'Bowling Alley',
 'Boxing Gym',
 'Brasserie',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bridal Shop',
 'Bridge',

In [32]:
f = vdf.groupby(['Borough','Neighborhood','Venue Category']).size()

In [147]:
#vdf1 = getNearbyVenues(newdf['Borough'],newdf['Neighborhood'], newdf['Latitude'], newdf['Longitude'], radius=2000)
vdf1 = pd.read_csv('vdf1.csv')

In [251]:
L = list(set(newdf.Neighborhood).difference(set(V500.Neighborhood)))

In [ ]:
newdf[newdf['Neighborhood']=='Tottenham']

In [574]:
vdf = pd.read_csv('vdf500.csv')
td = pd.get_dummies(vdf[['Venue Category']], prefix="", prefix_sep="")
td[['Neighborhood','Borough']] = vdf[['Neighborhood','Borough']] 
td_g = td.groupby(['Borough','Neighborhood']).mean().reset_index()
V500 = td_g.groupby(['Borough','Neighborhood']).mean()
V500.reset_index(inplace=True)
V500.tail()

,Borough,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Zoo Exhibit
512,Westminster,Soho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.010000,0.000000,0.0,0.010000,0.0,0.0,0.0,0.0
513,Westminster,St James's,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.015152,0.0,0.030303,0.0,0.0,0.0,0.0
514,Westminster,St John's Wood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
515,Westminster,Temple,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.015873,0.015873,0.0,0.000000,0.0,0.0,0.0,0.0
516,Westminster,Westminster,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [290]:
V500.Borough.unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
       'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster'],
      dtype=object)

In [591]:
newdf = pd.read_csv('newdf.csv')
newdf.reset_index(inplace=True)
V2001 = V2000.reset_index()
L = list(set(newdf.Neighborhood).difference(set(V2001.Neighborhood)))
newdf.drop(['index','Unnamed: 0'],axis=1,inplace=True)
newdf.set_index('Neighborhood',inplace=True)
newdf = newdf.drop(L)
newdf.reset_index(inplace=True)
newdf.shape

(517, 5)

In [592]:
T = ['Library','Gym','Italian Restaurant']
for i in T:
    newdf[i] = V500[i]
newdf

,Neighborhood,Borough,Latitude,Longitude,Rate,Library,Gym,Italian Restaurant
0,Abbey Wood,Bexley,51.489962,0.132891,45.6,0.0,0.052632,0.000000
1,Albany Park,Bexley,51.435384,0.125965,45.6,0.0,0.000000,0.000000
2,Barnehurst,Bexley,51.465021,0.159647,45.6,0.0,0.000000,0.000000
3,Barnes Cray,Bexley,51.453299,0.192024,45.6,0.0,0.000000,0.000000
4,Belvedere,Bexley,51.489473,0.147359,45.6,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
512,Sands End,Hammersmith and Fulham,51.472545,-0.187112,98.1,0.0,0.010000,0.000000
513,Shepherd's Bush,Hammersmith and Fulham,51.505314,-0.222901,98.1,0.0,0.000000,0.015152
514,West Kensington,Hammersmith and Fulham,51.490662,-0.205916,98.1,0.0,0.000000,0.000000
515,White City,Hammersmith and Fulham,51.511935,-0.224236,98.1,0.0,0.000000,0.015873


In [593]:
newdflist = list(map(lambda x: x+' 500',T))
newdf.columns = newdf.columns[:-len(T)].tolist()+newdflist
newdf.tail()

,Neighborhood,Borough,Latitude,Longitude,Rate,Library 500,Gym 500,Italian Restaurant 500
512,Sands End,Hammersmith and Fulham,51.472545,-0.187112,98.1,0.0,0.01,0.000000
513,Shepherd's Bush,Hammersmith and Fulham,51.505314,-0.222901,98.1,0.0,0.00,0.015152
514,West Kensington,Hammersmith and Fulham,51.490662,-0.205916,98.1,0.0,0.00,0.000000
515,White City,Hammersmith and Fulham,51.511935,-0.224236,98.1,0.0,0.00,0.015873
516,Wormwood Scrubs,Hammersmith and Fulham,51.521380,-0.240439,98.1,0.0,0.00,0.020408


In [594]:
vdf1 = pd.read_csv('vdf1.csv')
td1 = pd.get_dummies(vdf1[['Venue Category']], prefix="", prefix_sep="")
td1[['Neighborhood','Borough']] = vdf[['Neighborhood','Borough']] 
td1_g = td1.groupby(['Borough','Neighborhood']).mean().reset_index()
V2000 = td1_g.groupby(['Borough','Neighborhood']).mean()
V2000.reset_index(inplace=True)

In [595]:
rental = pd.read_csv('rental.csv')
rental1 = rental[rental['date']=='2019-12-01']
rental1 = rental1[['area','median_salary']]
print('discrepancies are',set(newdf.Borough).difference(set(rental1.area)))
rental1['area'] = rental1['area'].map(lambda x:x[0].upper()+x[1:])
rental1['area'] = rental1['area'].replace({'Barking and dagenham':'Barking and Dagenham','Kingston upon thames':'Kingston upon Thames',
                                          'Kensington and chelsea':'Kensington and Chelsea','Tower hamlets':'Tower Hamlets','Richmond upon thames':'Richmond upon Thames',
                                          'Hammersmith and fulham':'Hammersmith and Fulham','Waltham forest':'Waltham Forest'})
print('no more discrepancies',set(newdf.Borough).difference(set(rental1.area)))
rental1 = rental1[1:-18]
set(rental1.area).difference(set(newdf.Borough))
rental1.columns = ['Borough','Standard']
fill = rental[(rental['area']=='bromley') & (rental['date']=='2018-12-01')]['median_salary'].values[0]
rental1 = rental1.fillna(fill)#Only NA is in Bromley and we have filled it with past years data,
newdf= pd.merge(rental1,newdf,on='Borough')
newdf

discrepancies are {'Westminster', 'Lewisham', 'Bexley', 'Barking and Dagenham', 'Lambeth', 'Hillingdon', 'Tower Hamlets', 'Camden', 'Richmond upon Thames', 'Hammersmith and Fulham', 'Enfield', 'Newham', 'Ealing', 'Waltham Forest', 'Havering', 'Wandsworth', 'Kingston upon Thames', 'Islington', 'Greenwich', 'Kensington and Chelsea', 'Hounslow', 'Southwark', 'Croydon', 'Brent', 'Merton', 'Harrow', 'Redbridge', 'Haringey', 'Hackney', 'Bromley', 'Sutton', 'Barnet'}
no more discrepancies set()


,Borough,Standard,Neighborhood,Latitude,Longitude,Rate,Library 500,Gym 500,Italian Restaurant 500
0,Barking and Dagenham,28738.0,Barking,51.540268,0.079324,70.9,0.0,0.000000,0.000000
1,Barking and Dagenham,28738.0,Becontree,51.540311,0.126524,70.9,0.0,0.046512,0.023256
2,Barking and Dagenham,28738.0,Becontree Heath,51.561030,0.147879,70.9,0.0,0.000000,0.000000
3,Barking and Dagenham,28738.0,Castle Green,51.533404,0.120052,70.9,0.0,0.052632,0.026316
4,Barking and Dagenham,28738.0,Dagenham,51.541327,0.148114,70.9,0.0,0.000000,0.083333
...,...,...,...,...,...,...,...,...,...
512,Westminster,43556.0,Soho,51.513163,-0.131175,171.3,0.0,0.000000,0.000000
513,Westminster,43556.0,St James's,51.507908,-0.136573,171.3,0.0,0.000000,0.000000
514,Westminster,43556.0,St John's Wood,51.531726,-0.174190,171.3,0.0,0.000000,0.045455
515,Westminster,43556.0,Temple,51.510966,-0.114335,171.3,0.0,0.000000,0.000000


In [580]:
for i in T:
    newdf[i] = V2000[i]
newdf

,Borough,Standard,Neighborhood,Latitude,Longitude,Rate,Library 500,Gym 500,Italian Restaurant 500,Library,Gym,Italian Restaurant
0,Barking and Dagenham,28738.0,Barking,51.540268,0.079324,70.9,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,Barking and Dagenham,28738.0,Becontree,51.540311,0.126524,70.9,0.0,0.046512,0.023256,0.0,0.000000,0.000000
2,Barking and Dagenham,28738.0,Becontree Heath,51.561030,0.147879,70.9,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,Barking and Dagenham,28738.0,Castle Green,51.533404,0.120052,70.9,0.0,0.052632,0.026316,0.0,0.000000,0.000000
4,Barking and Dagenham,28738.0,Dagenham,51.541327,0.148114,70.9,0.0,0.000000,0.083333,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
512,Westminster,43556.0,Soho,51.513163,-0.131175,171.3,0.0,0.000000,0.000000,0.0,0.000000,0.040000
513,Westminster,43556.0,St James's,51.507908,-0.136573,171.3,0.0,0.000000,0.000000,0.0,0.015152,0.000000
514,Westminster,43556.0,St John's Wood,51.531726,-0.174190,171.3,0.0,0.000000,0.045455,0.0,0.000000,0.029412
515,Westminster,43556.0,Temple,51.510966,-0.114335,171.3,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [596]:
newdflist = list(map(lambda x: x+' 2000',T))
newdf.columns = newdf.columns[:-len(T)].tolist()+newdflist
newdf.tail()

,Borough,Standard,Neighborhood,Latitude,Longitude,Rate,Library 2000,Gym 2000,Italian Restaurant 2000
512,Westminster,43556.0,Soho,51.513163,-0.131175,171.3,0.0,0.0,0.000000
513,Westminster,43556.0,St James's,51.507908,-0.136573,171.3,0.0,0.0,0.000000
514,Westminster,43556.0,St John's Wood,51.531726,-0.174190,171.3,0.0,0.0,0.045455
515,Westminster,43556.0,Temple,51.510966,-0.114335,171.3,0.0,0.0,0.000000
516,Westminster,43556.0,Westminster,51.500444,-0.126540,171.3,0.0,0.0,0.000000


In [597]:
college = 'Imperial College, London'
location = geolocator.geocode(college)
clatitude = location.latitude
clongitude = location.longitude
print('Latitude and Longitude of this college is {},{}'.format(clatitude,clongitude))

Latitude and Longitude of this college is 51.4994284,-0.1797084


In [598]:
K = list(map(lambda x:1/(x**0.5),list(map(lambda x:x**2,newdf['Latitude']-clatitude))+list(map(lambda x:x**2,newdf['Longitude']-clongitude))))

In [604]:
l1 = np.array(list(map(lambda x:x**2,newdf['Latitude']-clatitude)))
l2 = np.array(list(map(lambda x:x**2,newdf['Longitude']-clongitude)))
#[sum(x) for x in zip(L)]
from sklearn.preprocessing import MinMaxScaler
std = MinMaxScaler()
T = 1/(np.sqrt(l1+l2))
newdf['distance']  = std.fit_transform(T.reshape(len(T),1))
newdf['Rate'] = std.fit_transform(newdf['Rate'].to_numpy().reshape(len(newdf),1))
newdf

,Borough,Standard,Neighborhood,Latitude,Longitude,Rate,Library 2000,Gym 2000,Italian Restaurant 2000,distance,label
0,Barking and Dagenham,28738.0,Barking,51.540268,0.079324,0.205067,0.0,0.000000,0.000000,0.015159,1
1,Barking and Dagenham,28738.0,Becontree,51.540311,0.126524,0.205067,0.0,0.046512,0.023256,0.010131,1
2,Barking and Dagenham,28738.0,Becontree Heath,51.561030,0.147879,0.205067,0.0,0.000000,0.000000,0.008067,1
3,Barking and Dagenham,28738.0,Castle Green,51.533404,0.120052,0.205067,0.0,0.052632,0.026316,0.010811,1
4,Barking and Dagenham,28738.0,Dagenham,51.541327,0.148114,0.205067,0.0,0.000000,0.083333,0.008292,3
...,...,...,...,...,...,...,...,...,...,...,...
512,Westminster,43556.0,Soho,51.513163,-0.131175,1.000000,0.0,0.000000,0.000000,0.154778,0
513,Westminster,43556.0,St James's,51.507908,-0.136573,1.000000,0.0,0.000000,0.000000,0.180250,0
514,Westminster,43556.0,St John's Wood,51.531726,-0.174190,1.000000,0.0,0.000000,0.045455,0.248021,0
515,Westminster,43556.0,Temple,51.510966,-0.114335,1.000000,0.0,0.000000,0.000000,0.113255,0


In [609]:
pKmeans = newdf.drop(['Neighborhood','Borough','Latitude','Longitude'],axis=1)
for i in pKmeans.columns:
    pKmeans[i] = std.fit_transform(pKmeans[i].to_numpy().reshape(len(pKmeans),1))
from sklearn.cluster import KMeans
clusters=4
km = KMeans(clusters)
km.fit(pKmeans)
newdf['label'] = km.labels_

In [610]:
city = 'London'
lc = geolocator.geocode(city)
clc = location.latitude
clo = location.longitude
print('Latitude and Longitude of this London is {},{}'.format(clc,clo))
newdf

Latitude and Longitude of this London is 51.4994284,-0.1797084


,Borough,Standard,Neighborhood,Latitude,Longitude,Rate,Library 2000,Gym 2000,Italian Restaurant 2000,distance,label
0,Barking and Dagenham,28738.0,Barking,51.540268,0.079324,0.205067,0.0,0.000000,0.000000,0.015159,1
1,Barking and Dagenham,28738.0,Becontree,51.540311,0.126524,0.205067,0.0,0.046512,0.023256,0.010131,1
2,Barking and Dagenham,28738.0,Becontree Heath,51.561030,0.147879,0.205067,0.0,0.000000,0.000000,0.008067,1
3,Barking and Dagenham,28738.0,Castle Green,51.533404,0.120052,0.205067,0.0,0.052632,0.026316,0.010811,1
4,Barking and Dagenham,28738.0,Dagenham,51.541327,0.148114,0.205067,0.0,0.000000,0.083333,0.008292,0
...,...,...,...,...,...,...,...,...,...,...,...
512,Westminster,43556.0,Soho,51.513163,-0.131175,1.000000,0.0,0.000000,0.000000,0.154778,3
513,Westminster,43556.0,St James's,51.507908,-0.136573,1.000000,0.0,0.000000,0.000000,0.180250,3
514,Westminster,43556.0,St John's Wood,51.531726,-0.174190,1.000000,0.0,0.000000,0.045455,0.248021,3
515,Westminster,43556.0,Temple,51.510966,-0.114335,1.000000,0.0,0.000000,0.000000,0.113255,3


In [611]:
map_clusters = folium.Map(location=[clc, clo], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.viridis(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
folium.CircleMarker(
    [clatitude,clongitude],
    radius=15,
    popup=college,
    color = 'Red',
    fill=True,
    fill_color='Red',
    fill_opacity=0.8).add_to(map_clusters)
# add markers to the map
markers_colors = []
for lat, lon,boro, poi, cluster in zip(newdf['Latitude'], newdf['Longitude'], newdf['Borough'],newdf['Neighborhood'], newdf['label']):
    label = folium.Popup(str(poi) + ', ' + str(boro), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters